# Khởi tạo trọng số (weight) và độ lệch (bias)

Trong bài đọc này, chúng tôi nghiên cứu các cách khác nhau để khởi tạo trọng số và độ lệch trong các lớp của mạng nơ-ron.

In [ ]:
%matplotlib inline
import tensorflow as tf
import pandas as pd
print(tf.__version__)

### Trọng số và bias mặc định

Trong các mô hình chúng ta đã làm cho đến nay, chúng ta vẫn chưa chỉ định các giá trị ban đầu của trọng số và bias trong mỗi lớp của mạng nơ-ron.

Các giá trị mặc định của trọng số và bias trong TensorFlow phụ thuộc vào loại lớp mà chúng ta đang sử dụng.

Ví dụ: trong lớp `Dense`, bias được đặt thành 0 (`số không`) theo mặc định, trong khi trọng số được đặt theo `glorot_uniform`.

Khởi tạo Glorot lấy ngẫu nhiên các trọng số một cách đồng nhất từ khoảng đóng $ [- c, c] $, trong đó $$c = \sqrt{\frac{6}{n_{input}+n_{output}}}$$

và $ n_ {input} $ và $ n_ {output} $ là số lượng đầu vào và đầu ra từ lớp tương ứng.

### Khởi tạo trọng số và bias
Chúng ta thường muốn khởi tạo trọng số và bias của mình, và TensorFlow làm cho quá trình này đơn giản hơn.

Khi chúng ta xây dựng một mô hình trong TensorFlow, mỗi lớp có các đối số tùy chọn `kernel_initialiser` và `bias_initialiser`, được sử dụng để đặt trọng số và bias tương ứng.

Nếu một lớp không có trọng số hoặc bias (ví dụ: lớp pooling layer tối đa), thì việc cố gắng đặt `kernel_initialiser` hoặc `bias_initialiser` sẽ gây ra lỗi.

Hãy xem ví dụ sử dụng một số cách khởi tạo khác nhau có sẵn trong Keras.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPooling1D 

In [ ]:
#Constructing a model

model = Sequential([
    Conv1D(filters=16, kernel_size=3, input_shape=(128, 64), kernel_initializer='random_uniform', bias_initializer="zeros", activation='relu'),
    MaxPooling1D(pool_size=4),
    Flatten(),
    Dense(64, kernel_initializer='he_uniform', bias_initializer='ones', activation='relu'),
])

Như ví dụ minh họa sau, chúng ta cũng có thể khởi tạo các trình khởi tạo theo một cách hơi khác cho phép đặt các đối số tùy chọn của phương thức khởi tạo.

In [ ]:
#Adding some layers to our model

model.add(Dense(64, 
                kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                bias_initializer=tf.keras.initializers.Constant(value=0.4), 
                activation='relu'),)

model.add(Dense(8, 
                kernel_initializer=tf.keras.initializers.Orthogonal(gain=1.0, seed=None), 
                bias_initializer=tf.keras.initializers.Constant(value=0.4), 
                activation='relu'))

### Khởi tạo trọng số và bias tùy chỉnh 
Bạn cũng có thể xác định trọng số của mình và các khởi tạo bias.
Các trình khởi tạo phải có 2 đối số, `shape` của tensor được khởi tạo và `dtype`.

Đây là một ví dụ nhỏ cho thấy cách bạn có thể sử dụng trình khởi tạo tùy chỉnh của mình trong một lớp.

In [ ]:
import tensorflow.keras.backend as K

In [ ]:
#Example of a custom initializer

def my_init(shape, dtype=None):
    return K.random_normal(shape, dtype=dtype)

model.add(Dense(64, kernel_initializer=my_init))

Hãy xem tóm tắt về mô hình đã hoàn thiện của chúng ta.

In [ ]:
model.summary()

### Trực quan hóa trọng số và bias khởi tạo

Cuối cùng, có thể thấy hiệu quả của các trình khởi tạo đối với trọng số và bias bằng cách vẽ biểu đồ của các giá trị kết quả. So sánh các biểu đồ này với các trình khởi tạo đã chọn cho mỗi lớp ở trên.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(12,16))
fig.subplots_adjust(hspace=0.5, wspace=0.5)

# Filter out the pooling and flatten layers, that don't have any weights
weight_layers = [layer for layer in model.layers if len(layer.weights) > 0]

for i, layer in enumerate(weight_layers):
    for j in [0, 1]:
        axes[i, j].hist(layer.weights[j].numpy().flatten(), align='left')
        axes[i, j].set_title(layer.weights[j].name)

## Nguồn tài liệu đọc thêm
* https://keras.io/initializers/
* https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/initializers